In [50]:
import pandas as pd
import numpy as np

In [51]:
data = [['20-10-2020', 'PSG', 'Man U', 1, 2], ['20-10-2020', 'Leipzig','Istanbul',2,0], ['27-10-2020', 'Istanbul','PSG',0,2], ['27-10-2020', 'Man U','Leipzig',5,0]] 
df = pd.DataFrame(data, columns = ['Date', 'Home', 'Away', 'HG', 'AG']) 

In [52]:
df

,Date,Home,Away,HG,AG
0,20-10-2020,PSG,Man U,1,2
1,20-10-2020,Leipzig,Istanbul,2,0
2,27-10-2020,Istanbul,PSG,0,2
3,27-10-2020,Man U,Leipzig,5,0


In [53]:
df_home = df.reset_index(level=0)
columns = {
    "Date": 'date',
    "Home": "team",
    "Away": "opponent",
    'HG': 'team_goals',
    'AG': 'opponent_goals',
}
df_home = df_home.rename(columns=columns)
df_home['site'] = 'H'

df_away = df.reset_index(level=0)
columns = {
    "Date": 'date',
    "Home": "opponent",
    "Away": "team",
    'HG': 'opponent_goals',
    'AG': 'team_goals',
}
df_away = df_away.rename(columns=columns)
df_away['site'] = 'A'

df_team = pd.concat([df_home, df_away], ignore_index=True).sort_values(['date'])
df_team['team'] = df_team['team'].astype('category')
df_team['opponent'] = df_team['opponent'].astype('category')

C:\Users\SDEGOSSONDEVARENNE\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [54]:
df_team

,date,index,opponent,opponent_goals,site,team,team_goals
0,20-10-2020,0,Man U,2,H,PSG,1
1,20-10-2020,1,Istanbul,0,H,Leipzig,2
4,20-10-2020,0,PSG,1,A,Man U,2
5,20-10-2020,1,Leipzig,2,A,Istanbul,0
2,27-10-2020,2,PSG,2,H,Istanbul,0
3,27-10-2020,3,Leipzig,0,H,Man U,5
6,27-10-2020,2,Istanbul,0,A,PSG,2
7,27-10-2020,3,Man U,5,A,Leipzig,0


In [55]:
conditions = [df_team['team_goals'] > df_team['opponent_goals'], df_team['team_goals'] == df_team['opponent_goals']]
choices = [3, 1]
df_team['pts'] = np.select(conditions, choices, default=0)
f = lambda x: x.shift(1).rolling(1).sum()
df_team['form_l1_before'] = df_team.groupby(['team'])['pts'].apply(f)
df_team['goal_l1_before'] = df_team.groupby(['team'])['team_goals'].apply(f)
print(df_team)

         date  index  opponent  opponent_goals site      team  team_goals  \
0  20-10-2020      0     Man U               2    H       PSG           1   
1  20-10-2020      1  Istanbul               0    H   Leipzig           2   
4  20-10-2020      0       PSG               1    A     Man U           2   
5  20-10-2020      1   Leipzig               2    A  Istanbul           0   
2  27-10-2020      2       PSG               2    H  Istanbul           0   
3  27-10-2020      3   Leipzig               0    H     Man U           5   
6  27-10-2020      2  Istanbul               0    A       PSG           2   
7  27-10-2020      3     Man U               5    A   Leipzig           0   

   pts  form_l1_before  goal_l1_before  
0    0             NaN             NaN  
1    3             NaN             NaN  
4    3             NaN             NaN  
5    0             NaN             NaN  
2    0             0.0             0.0  
3    3             3.0             2.0  
6    3             

In [62]:
p = df_team.pivot(index='opponent', columns=['site', 'team'], values='team_goals')

KeyError: 'Level site not found'

In [60]:
df_team2

,opponent,site,variable,value
0,Man U,H,date,20-10-2020
1,Istanbul,H,date,20-10-2020
2,PSG,A,date,20-10-2020
3,Leipzig,A,date,20-10-2020
4,PSG,H,date,27-10-2020
...,...,...,...,...
59,Leipzig,A,goal_l1_before,NaN
60,PSG,H,goal_l1_before,0
61,Leipzig,H,goal_l1_before,2
62,Istanbul,A,goal_l1_before,1
